This is version 9.2, featuring only DCGAN

### Libraries

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Keras modules
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, save_img

### Data preparation

In [9]:
pneumonia_dataset_dir = 'chest_xray/train/PNEUMONIA'
pneumonia_images = os.listdir(pneumonia_dataset_dir)

def get_data(data_path):
    x = []
    
    for filename in data_path:
        img = img_to_array(load_img(pneumonia_dataset_dir + "/" + filename, target_size=(28,28,3)))
        x.append(img)
        
    x = np.array(x).astype('float32')
    x = (x - 127.5) / 127.5
    
    return x

pneumonia_dataset = get_data(pneumonia_images)

### Define DC GAN

In [14]:
class DCGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.history = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense((128*7*7), activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((7,7,128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation('tanh'))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        #X_train = normal_dataset
        X_train = pneumonia_dataset

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, D acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
            self.history.append({"D": d_loss[0], "G": g_loss})
    
    def _predict_noise(self, size):
        noise = np.random.normal(0, 1, (size, self.latent_dim))
        return self.generator.predict(noise)
        
    def gen(self, size):
        generated = self._predict_noise(size)
        generated = 0.5 * generated + 0.5
        return generated

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("output/%d_pneumonia.png" % epoch)
        plt.close()
    
    def plot_loss(self):
        hist = pd.DataFrame(self.history)
        plt.figure(figsize=(20,5))
        for colnm in hist.columns:
            plt.plot(hist[colnm],label=colnm)
        plt.legend()
        plt.ylabel("loss")
        plt.xlabel("epochs")
        plt.show()

In [ ]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=10000, batch_size=32, sample_interval=500)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 14, 14, 32)        896       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 8, 8, 64)         

63 [D loss: 0.147075, D acc.: 95.31%] [G loss: 3.245283]
64 [D loss: 0.091895, D acc.: 98.44%] [G loss: 2.322093]
65 [D loss: 0.130595, D acc.: 96.88%] [G loss: 2.681814]
66 [D loss: 0.127128, D acc.: 96.88%] [G loss: 2.962873]
67 [D loss: 0.139677, D acc.: 96.88%] [G loss: 2.264380]
68 [D loss: 0.071049, D acc.: 100.00%] [G loss: 3.237006]
69 [D loss: 0.322705, D acc.: 82.81%] [G loss: 2.009206]
70 [D loss: 0.063029, D acc.: 100.00%] [G loss: 3.239723]
71 [D loss: 0.203324, D acc.: 92.19%] [G loss: 2.317349]
72 [D loss: 0.070942, D acc.: 100.00%] [G loss: 2.147605]
73 [D loss: 0.076090, D acc.: 98.44%] [G loss: 1.826568]
74 [D loss: 0.048102, D acc.: 100.00%] [G loss: 3.406927]
75 [D loss: 0.300353, D acc.: 89.06%] [G loss: 2.195366]
76 [D loss: 0.119773, D acc.: 96.88%] [G loss: 2.313711]
77 [D loss: 0.086974, D acc.: 96.88%] [G loss: 1.918263]
78 [D loss: 0.065219, D acc.: 100.00%] [G loss: 2.994929]
79 [D loss: 0.053107, D acc.: 100.00%] [G loss: 2.679245]
80 [D loss: 0.036096, D a

205 [D loss: 0.002279, D acc.: 100.00%] [G loss: 1.150086]
206 [D loss: 0.003998, D acc.: 100.00%] [G loss: 1.199035]
207 [D loss: 0.001146, D acc.: 100.00%] [G loss: 1.180697]
208 [D loss: 0.001998, D acc.: 100.00%] [G loss: 1.026770]
209 [D loss: 0.002702, D acc.: 100.00%] [G loss: 0.906831]
210 [D loss: 0.002300, D acc.: 100.00%] [G loss: 0.956424]
211 [D loss: 0.002332, D acc.: 100.00%] [G loss: 0.926009]
212 [D loss: 0.002959, D acc.: 100.00%] [G loss: 1.249742]
213 [D loss: 0.002965, D acc.: 100.00%] [G loss: 1.148209]
214 [D loss: 0.002277, D acc.: 100.00%] [G loss: 1.436892]
215 [D loss: 0.013412, D acc.: 100.00%] [G loss: 1.294437]
216 [D loss: 0.003637, D acc.: 100.00%] [G loss: 1.551599]
217 [D loss: 0.003381, D acc.: 100.00%] [G loss: 0.744566]
218 [D loss: 0.002687, D acc.: 100.00%] [G loss: 0.788294]
219 [D loss: 0.005321, D acc.: 100.00%] [G loss: 1.187356]
220 [D loss: 0.003274, D acc.: 100.00%] [G loss: 0.862219]
221 [D loss: 0.002778, D acc.: 100.00%] [G loss: 0.93707

345 [D loss: 0.000310, D acc.: 100.00%] [G loss: 0.480684]
346 [D loss: 0.000761, D acc.: 100.00%] [G loss: 1.000230]
347 [D loss: 0.000563, D acc.: 100.00%] [G loss: 0.611677]
348 [D loss: 0.000945, D acc.: 100.00%] [G loss: 0.696793]
349 [D loss: 0.000527, D acc.: 100.00%] [G loss: 0.755435]
350 [D loss: 0.000222, D acc.: 100.00%] [G loss: 0.822888]
351 [D loss: 0.000336, D acc.: 100.00%] [G loss: 0.817652]
352 [D loss: 0.000973, D acc.: 100.00%] [G loss: 0.804296]
353 [D loss: 0.003019, D acc.: 100.00%] [G loss: 0.557117]
354 [D loss: 0.000439, D acc.: 100.00%] [G loss: 0.657127]
355 [D loss: 0.000919, D acc.: 100.00%] [G loss: 0.682068]
356 [D loss: 0.000972, D acc.: 100.00%] [G loss: 0.707266]
357 [D loss: 0.000443, D acc.: 100.00%] [G loss: 0.794855]
358 [D loss: 0.000319, D acc.: 100.00%] [G loss: 0.573162]
359 [D loss: 0.000359, D acc.: 100.00%] [G loss: 0.826278]
360 [D loss: 0.000486, D acc.: 100.00%] [G loss: 0.828991]
361 [D loss: 0.000243, D acc.: 100.00%] [G loss: 0.50582

485 [D loss: 0.000184, D acc.: 100.00%] [G loss: 0.453803]
486 [D loss: 0.000218, D acc.: 100.00%] [G loss: 0.568691]
487 [D loss: 0.000318, D acc.: 100.00%] [G loss: 0.465893]
488 [D loss: 0.000357, D acc.: 100.00%] [G loss: 0.415803]
489 [D loss: 0.000145, D acc.: 100.00%] [G loss: 0.392200]
490 [D loss: 0.000191, D acc.: 100.00%] [G loss: 0.474132]
491 [D loss: 0.000167, D acc.: 100.00%] [G loss: 0.404289]
492 [D loss: 0.000343, D acc.: 100.00%] [G loss: 0.294563]
493 [D loss: 0.000124, D acc.: 100.00%] [G loss: 0.581380]
494 [D loss: 0.000206, D acc.: 100.00%] [G loss: 0.376355]
495 [D loss: 0.000414, D acc.: 100.00%] [G loss: 0.511281]
496 [D loss: 0.000341, D acc.: 100.00%] [G loss: 0.503982]
497 [D loss: 0.000234, D acc.: 100.00%] [G loss: 0.397968]
498 [D loss: 0.000211, D acc.: 100.00%] [G loss: 0.440587]
499 [D loss: 0.000146, D acc.: 100.00%] [G loss: 0.380180]
500 [D loss: 0.000252, D acc.: 100.00%] [G loss: 0.462833]
501 [D loss: 0.000240, D acc.: 100.00%] [G loss: 0.41930

...

2147 [D loss: 0.000065, D acc.: 100.00%] [G loss: 0.278728]
2148 [D loss: 0.000250, D acc.: 100.00%] [G loss: 0.341797]
2149 [D loss: 0.000127, D acc.: 100.00%] [G loss: 0.359525]
2150 [D loss: 0.000114, D acc.: 100.00%] [G loss: 0.211925]
2151 [D loss: 0.000093, D acc.: 100.00%] [G loss: 0.335868]
2152 [D loss: 0.000058, D acc.: 100.00%] [G loss: 0.371546]
2153 [D loss: 0.000238, D acc.: 100.00%] [G loss: 0.328052]
2154 [D loss: 0.000338, D acc.: 100.00%] [G loss: 0.496213]
2155 [D loss: 0.000174, D acc.: 100.00%] [G loss: 0.322626]
2156 [D loss: 0.000085, D acc.: 100.00%] [G loss: 0.310327]
2157 [D loss: 0.000146, D acc.: 100.00%] [G loss: 0.297561]
2158 [D loss: 0.000137, D acc.: 100.00%] [G loss: 0.386426]
2159 [D loss: 0.000065, D acc.: 100.00%] [G loss: 0.207010]
2160 [D loss: 0.000239, D acc.: 100.00%] [G loss: 0.177958]
2161 [D loss: 0.000121, D acc.: 100.00%] [G loss: 0.253783]
2162 [D loss: 0.000189, D acc.: 100.00%] [G loss: 0.235376]
2163 [D loss: 0.000123, D acc.: 100.00%]

2285 [D loss: 0.000051, D acc.: 100.00%] [G loss: 0.126946]
2286 [D loss: 0.000127, D acc.: 100.00%] [G loss: 0.132189]
2287 [D loss: 0.000116, D acc.: 100.00%] [G loss: 0.219402]
2288 [D loss: 0.000056, D acc.: 100.00%] [G loss: 0.189967]
2289 [D loss: 0.000056, D acc.: 100.00%] [G loss: 0.237690]
2290 [D loss: 0.000041, D acc.: 100.00%] [G loss: 0.220563]
2291 [D loss: 0.000183, D acc.: 100.00%] [G loss: 0.252147]
2292 [D loss: 0.000064, D acc.: 100.00%] [G loss: 0.202885]
2293 [D loss: 0.000093, D acc.: 100.00%] [G loss: 0.168379]
2294 [D loss: 0.000054, D acc.: 100.00%] [G loss: 0.083259]
2295 [D loss: 0.000039, D acc.: 100.00%] [G loss: 0.127349]
2296 [D loss: 0.000050, D acc.: 100.00%] [G loss: 0.123372]
2297 [D loss: 0.000093, D acc.: 100.00%] [G loss: 0.101928]
2298 [D loss: 0.000064, D acc.: 100.00%] [G loss: 0.132303]
2299 [D loss: 0.000029, D acc.: 100.00%] [G loss: 0.165610]
2300 [D loss: 0.000092, D acc.: 100.00%] [G loss: 0.122499]
2301 [D loss: 0.000055, D acc.: 100.00%]

2423 [D loss: 0.000069, D acc.: 100.00%] [G loss: 0.152711]
2424 [D loss: 0.000022, D acc.: 100.00%] [G loss: 0.212667]
2425 [D loss: 0.000041, D acc.: 100.00%] [G loss: 0.168183]
2426 [D loss: 0.000034, D acc.: 100.00%] [G loss: 0.313548]
2427 [D loss: 0.000069, D acc.: 100.00%] [G loss: 0.236436]
2428 [D loss: 0.000049, D acc.: 100.00%] [G loss: 0.229959]
2429 [D loss: 0.000014, D acc.: 100.00%] [G loss: 0.264732]
2430 [D loss: 0.000017, D acc.: 100.00%] [G loss: 0.211678]
2431 [D loss: 0.000021, D acc.: 100.00%] [G loss: 0.200925]
2432 [D loss: 0.000035, D acc.: 100.00%] [G loss: 0.283984]
2433 [D loss: 0.000018, D acc.: 100.00%] [G loss: 0.090519]
2434 [D loss: 0.000013, D acc.: 100.00%] [G loss: 0.173665]
2435 [D loss: 0.000012, D acc.: 100.00%] [G loss: 0.079882]
2436 [D loss: 0.000016, D acc.: 100.00%] [G loss: 0.183746]
2437 [D loss: 0.000016, D acc.: 100.00%] [G loss: 0.199930]
2438 [D loss: 0.000009, D acc.: 100.00%] [G loss: 0.254346]
2439 [D loss: 0.000018, D acc.: 100.00%]

2561 [D loss: 0.000104, D acc.: 100.00%] [G loss: 0.458281]
2562 [D loss: 0.000012, D acc.: 100.00%] [G loss: 0.177024]
2563 [D loss: 0.000020, D acc.: 100.00%] [G loss: 0.248962]
2564 [D loss: 0.000008, D acc.: 100.00%] [G loss: 0.128144]
2565 [D loss: 0.000120, D acc.: 100.00%] [G loss: 0.197568]
2566 [D loss: 0.000032, D acc.: 100.00%] [G loss: 0.174486]
2567 [D loss: 0.000052, D acc.: 100.00%] [G loss: 0.285404]
2568 [D loss: 0.000023, D acc.: 100.00%] [G loss: 0.226208]
2569 [D loss: 0.000031, D acc.: 100.00%] [G loss: 0.331457]
2570 [D loss: 0.000025, D acc.: 100.00%] [G loss: 0.173318]
2571 [D loss: 0.000015, D acc.: 100.00%] [G loss: 0.152233]
2572 [D loss: 0.000023, D acc.: 100.00%] [G loss: 0.169701]
2573 [D loss: 0.000013, D acc.: 100.00%] [G loss: 0.107375]
2574 [D loss: 0.000005, D acc.: 100.00%] [G loss: 0.071830]
2575 [D loss: 0.000011, D acc.: 100.00%] [G loss: 0.177975]
2576 [D loss: 0.000016, D acc.: 100.00%] [G loss: 0.121761]
2577 [D loss: 0.000062, D acc.: 100.00%]

2699 [D loss: 0.000013, D acc.: 100.00%] [G loss: 0.116337]
2700 [D loss: 0.000013, D acc.: 100.00%] [G loss: 0.120193]
2701 [D loss: 0.000003, D acc.: 100.00%] [G loss: 0.093336]
2702 [D loss: 0.000010, D acc.: 100.00%] [G loss: 0.099360]
2703 [D loss: 0.000006, D acc.: 100.00%] [G loss: 0.153065]
2704 [D loss: 0.000010, D acc.: 100.00%] [G loss: 0.142416]
2705 [D loss: 0.000030, D acc.: 100.00%] [G loss: 0.179620]
2706 [D loss: 0.000013, D acc.: 100.00%] [G loss: 0.169180]
2707 [D loss: 0.000014, D acc.: 100.00%] [G loss: 0.281354]
2708 [D loss: 0.000010, D acc.: 100.00%] [G loss: 0.145488]
2709 [D loss: 0.000003, D acc.: 100.00%] [G loss: 0.135374]
2710 [D loss: 0.000019, D acc.: 100.00%] [G loss: 0.199172]
2711 [D loss: 0.000014, D acc.: 100.00%] [G loss: 0.160289]
2712 [D loss: 0.000008, D acc.: 100.00%] [G loss: 0.175564]
2713 [D loss: 0.000008, D acc.: 100.00%] [G loss: 0.175974]
2714 [D loss: 0.000008, D acc.: 100.00%] [G loss: 0.060591]
2715 [D loss: 0.000008, D acc.: 100.00%]

In [ ]:
dcgan.plot_loss()

In [ ]:
gan_images = dcgan.gen(4273)

for i in range(len(gan_images)):
    #save_img("output/gan_normal/{}.png".format(i), gan_images[i])
    save_img("output/gan_pneumonia/{}.png".format(i+3000), gan_images[i])